In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class DSU:
  def __init__(self, n):
    self.P = list(range(n))
    self.s = [1] * n
    self.ns = n

  def find(self, a):
    _a = a
    while a != self.P[a]:
      a = self.P[a]
    while _a != a:
      self.P[_a], _a = a, self.P[_a]
    return a

  def union(self, a, b):
    a, b = self.find(a), self.find(b)
    if a != b:
      if self.s[a] < self.s[b]:
        a, b = b, a

      self.ns -= 1
      self.P[b] = a
      self.s[a] += self.s[b]

def sol() :
  N, M, K = map(int, input().split())
  L = [*map(int, input().split())]
  dsu = DSU(N)
  for _ in range(M):
    a, b = map(int, input().split())
    dsu.union(a-1, b-1)
  
  GL = {} #친구집단 id: [사탕의 총합, 집단의 크기] O(N)
  for i in range(N):
    id = dsu.find(i)
    if id not in GL:
      GL[id] = [L[i], dsu.s[id]]
    else:
      GL[id][0] += L[i]
  
  W, V = [], []
  for i, (v, w) in enumerate(GL.values()):
    W.append(w)
    V.append(v)

  dp = [[0] * (K+1) for _ in range(len(W))]
  for i in range(len(W)) :
    for j in range(K) :
      if j < W[i] :
        dp[i][j] = dp[i-1][j]
      else :
        dp[i][j] = max(dp[i-1][j], dp[i-1][j-W[i]] + V[i])

  print(max(dp[-1]))

sol()

### 자력솔
- 문제를 읽고 서브트리의 개수 + 그리디로 접근했다. 조금더 읽어보니 유니온 파인드를 쓰는게 더 맞는 것 같았다.(지문의 `친구의 친구는 친구다`)
- 유니온 파인드로 구해놓고 보니, 각 친구 그룹의 크기가 무게, 친구 그룹에 속한 사탕의 수의 합이 가치가 되는 배낭문제로 되는것이 보였다.
- 배낭문제 1차원 최적화를 해야 Python으로 풀리는 것 같았다. PyPy로 3272ms로 겨우 뚫었다.

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class DSU:
  def __init__(self, n):
    self.P = list(range(n))
    self.s = [1] * n
    self.ns = n

  def find(self, a):
    _a = a
    while a != self.P[a]:
      a = self.P[a]
    while _a != a:
      self.P[_a], _a = a, self.P[_a]
    return a

  def union(self, a, b):
    a, b = self.find(a), self.find(b)
    if a != b:
      if self.s[a] < self.s[b]:
        a, b = b, a

      self.ns -= 1
      self.P[b] = a
      self.s[a] += self.s[b]

def sol() :
  N, M, K = map(int, input().split())
  L = [*map(int, input().split())]
  dsu = DSU(N)
  for _ in range(M):
    a, b = map(int, input().split())
    dsu.union(a-1, b-1)
  
  GL = {} #친구집단 id: [사탕의 총합, 집단의 크기] O(N)
  for i in range(N):
    id = dsu.find(i)
    if id not in GL:
      GL[id] = [L[i], dsu.s[id]]
    else:
      GL[id][0] += L[i]
  
  W, V = [], []
  for i, (v, w) in enumerate(GL.values()):
    W.append(w)
    V.append(v)

  #0-1 knapsack
  dp = [0] * (K+1)
  cur = 0
  for i in range(len(W)):
    w, v = W[i], V[i]
    cur = min(K-1, cur + w)
    for j in range(cur, w-1, -1):
      dp[j] = max(dp[j], dp[j-w]+v)

  print(max(dp))

sol()

- knapsack 부분 최적화 버전